## Load required packages

In [1]:
import torch
from transformer.utils import create_tranformer_model
from transformer.utils import load_config, get_dataset, get_tokenizer
from transformer.utils import get_checkpoint_path
from transformer.engine import inference

## Load configuration

In [2]:
# Assign the device for computation (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
## Get Config, dataset, tokenizers
config_file_path = "./config.json"
config = load_config(config_file_path)
dataset = get_dataset(config)
tokenizer_src = get_tokenizer(config, dataset, config['language_source'])
tokenizer_tgt = get_tokenizer(config, dataset, config['language_target'])

## Define model, random input and get random output
# Define the dimensions and vocabulary size
# Define the parameters for the transformer model
vocab_size_src = tokenizer_src.get_vocab_size()
vocab_size_tgt = tokenizer_tgt.get_vocab_size()
seq_len = config['seq_len']


Using cuda device


## Load model and latest checkpoint

In [3]:
# Create the Transformer model
model = create_tranformer_model(config, vocab_size_src, vocab_size_tgt).to(device)
# Load latest checkpoint
model_checkpoint = get_checkpoint_path(config)
print("model_checkpoint:", model_checkpoint)
state = torch.load(model_checkpoint)
# Assign latest checkpoint to transfromer
model.load_state_dict(state['model_state_dict'])

model_checkpoint: models\transformer_16.pt


C:\Users\easyb\AppData\Local\Temp\ipykernel_11604\2278621582.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(model_checkpoint)


<All keys matched successfully>

## Translation 

In [4]:
source_text = "Market prices are shaped by the balance of supply and demand."
translated_text = inference(source_text, 
                            model, 
                            tokenizer_src, 
                            tokenizer_tgt,
                            seq_len,
                            device)

print("Source text: \n", source_text)
print(">>>>")
print("Translated text: \n", translated_text)

Source text: 
 Market prices are shaped by the balance of supply and demand.
>>>>
Translated text: 
 Les prix du marché sont façonnés par l ' équilibre de l ' offre et de la demande .


In [5]:
# Check token availability of a word
print("Encoding \'shaped\'", tokenizer_src.encode('shaped' ).ids)
print("Encoding \'déterminés\'", tokenizer_tgt.encode('déterminés' ).ids)

Encoding 'shaped' [3621]
Encoding 'déterminés' [4343]
